In [1]:
import os
import streamlit as st
import time
import pickle

from langchain_community.llms import Ollama
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

In [2]:
from langchain_community.llms import Ollama
llm = Ollama(model='mistral')

C:\Users\Somaa\AppData\Local\Temp\ipykernel_15144\3334827828.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model='mistral')


In [3]:
response = llm.invoke("hi how are you ?")
print(response)

 Hello! I'm an AI, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [4]:
# loading the data using UnstructuredURLLoader
from langchain.document_loaders import UnstructuredURLLoader

urls = [
    "https://www.moneycontrol.com/",
    "https://economictimes.indiatimes.com/?from=mdr"
]

loaders = UnstructuredURLLoader(urls=urls)

data = loaders.load()
len(data)

2

In [5]:
# now we will split the data into chunks using recursivecharsplitter

from langchain.text_splitter import RecursiveCharacterTextSplitter

re_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n","\n"," ", "."],
    chunk_size = 1000,
    chunk_overlap = 300,
    length_function = len
    
)

In [6]:
chunks = re_splitter.split_documents(data)

In [7]:
len(chunks)

170

In [ ]:
chunks[171]

In [ ]:
for chunk in chunks :
    print(chunk)
    print("\n" *3)

page_content='Close Ad

English

Hindi

Gujarati

Specials

Hello, Login

Hello, Login

Log-in or Sign-Up

My Account

My Profile

My Portfolio

My Watchlist

My Alerts

My Messages

Price Alerts

My Profile

My PRO

My Portfolio

My Watchlist

My Alerts

My Messages

Price Alerts

Logout

Loans up to ₹50 LAKHS

Fixed Deposits

Credit CardsLifetime Free

Credit Score

Chat with Us

Download App

Follow us on:

Network 18

Go Ad-Free

My Alerts

Moneycontrol

Go PRO Now PRO

Moneycontrol PRO

Monsoon Tracker

Web Stories

Crypto Tracker

Bank Holidays

Real Estate

Credit Cards Lifetime Free

Credit Score

IPO

Loans

MF Simplified

#NKSK

Get App Open In App

Moneycontrol

Enjoy seamless experience on APP

10x faster & More Feature

Get App

Moneycontrol

Moneycontrol PRO

Go PRO Now

Open App

Get App / Open App

Login

Sign-Up

English

हिंदी

ગુજરાતી

MC Login User Name

My Profile

My PRO

My Portfolio

My Watchlist

Loans @ 12%Up to ₹ 15 Lakhs!

Fixed Deposits

My Alerts

My Messa

In [ ]:
# now we have created chunks of our documents now we will proceed to make vector embeddings 

In [ ]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("all-mpnet-base-v2")

d:\Langchain_project\venv.rockyfinance\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# cant pass docs directly into sentence transformers , must extract the text content first
texts = [chunk.page_content for chunk in chunks]


In [ ]:
texts

['Close Ad\n\nEnglish\n\nHindi\n\nGujarati\n\nSpecials\n\nHello, Login\n\nHello, Login\n\nLog-in or Sign-Up\n\nMy Account\n\nMy Profile\n\nMy Portfolio\n\nMy Watchlist\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nMy Profile\n\nMy PRO\n\nMy Portfolio\n\nMy Watchlist\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nLogout\n\nLoans up to ₹50 LAKHS\n\nFixed Deposits\n\nCredit CardsLifetime Free\n\nCredit Score\n\nChat with Us\n\nDownload App\n\nFollow us on:\n\nNetwork 18\n\nGo Ad-Free\n\nMy Alerts\n\nMoneycontrol\n\nGo PRO Now PRO\n\nMoneycontrol PRO\n\nMonsoon Tracker\n\nWeb Stories\n\nCrypto Tracker\n\nBank Holidays\n\nReal Estate\n\nCredit Cards Lifetime Free\n\nCredit Score\n\nIPO\n\nLoans\n\nMF Simplified\n\n#NKSK\n\nGet App Open In App\n\nMoneycontrol\n\nEnjoy seamless experience on APP\n\n10x faster & More Feature\n\nGet App\n\nMoneycontrol\n\nMoneycontrol PRO\n\nGo PRO Now\n\nOpen App\n\nGet App / Open App\n\nLogin\n\nSign-Up\n\nEnglish\n\nहिंदी\n\nગુજરાતી\n\nMC Login User Name\n\

In [ ]:
vectors = encoder.encode(texts)

In [ ]:
dim =vectors.shape[1]
dim

768

In [ ]:
# creating an empty index using faiss
import faiss

index = faiss.IndexFlatL2(dim)

index.add(vectors)

In [ ]:
# Storing vector index create in local
file_path="vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(index, f)

In [ ]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# embeddings wrapper
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# directly create vectorstore
vectorstore = FAISS.from_texts(texts, embedding=embeddings)


# save
vectorstore.save_local("faiss_index")

# load
vectorstore = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)


# query
docs = vectorstore.similarity_search("what is current economic standing of India and USA?", k=2)
for d in docs:
    print(d.page_content)


India-US 6th round of trade talks scheduled for August 25 deferred

Next-Gen GST step towards a single tax slab GST

'Trump didn’t lose, Putin clearly won': Ex-US NSA John Bolton on Alaska summit

See More News

Watch

Trump-Putin Meet: Ukraine’s Voice Central In Ceasefire Talks, Says Zelenskyy

Trump-Putin Meet: Ukraine’s Voice Central In Ceasefire Talks, Says Zelenskyy Watch more

Russia Ukraine War: Ukraine’s Interceptor Drones, Low Cost Defense Against Russia’s Drone War | N18G

Russia Ukraine War: Ukraine’s Interceptor Drones, Low Cost Defense Against Russia’s Drone War | N18G Watch more

Data Story: Can RBI iron out these loan wrinkles?

Data Story: Can RBI iron out these loan wrinkles? Watch more

How to fix fixed income allocations amid rate cuts? Real rates hold the key

How to fix fixed income allocations amid rate cuts? Real rates hold the key Watch more

US overtakes Singapore to regain top spot with largest India equity holding
rise image 1

"No agreements that would go ag

In [ ]:
from langchain.chains import RetrievalQA
retriever = vectorstore.as_retriever(search_kwargs={'k':2})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever = retriever,
    return_source_documents =True
    
)


In [ ]:
query ="what is Donald Trumph's stand on the tarrif situation on different countries , imposed by USA ?"
result = qa_chain({'query':query})
print("Answer",result['result'])
print("sources")
for doc in result["source_documents"]:
    print("-",doc.page_content[:200])

Answer  In the provided context, there are two articles related to Donald Trump's tariffs. Here's a summary of his stand based on those articles:

1. "Trump’s tariffs hit India: What it means for GDP, debt and equity markets": The article does not specify any direct quotes from President Trump, but it mentions that Trump's tariffs have affected India. It implies that the tariffs have a negative impact on India's economy, affecting its GDP, debt, and equity markets.

2. "Trump Hints at No Extra Levy on India for Russian Oil Buy": In this article, President Trump is quoted as indicating that the US may not impose secondary tariffs on countries continuing to procure Russian crude oil. This suggests that, in the context of buying Russian oil, he may be willing to exempt certain countries from additional tariffs imposed by the USA.

It's important to note that this is based on the information provided in the articles and does not necessarily reflect Trump's overall stance on tariffs or trad

In [ ]:
query = "what is the India's GDP?"
# query = "what are the main features of punch iCNG?"

result = qa_chain({"query":query})



In [ ]:
print(result)

{'query': "what is the India's GDP?", 'result': " The provided context does not explicitly state the current GDP of India. However, it mentions that the tariffs imposed by Trump may have an impact on India's GDP, so it can be assumed that the GDP was already known before the imposition of these tariffs. Without specific numerical information, I cannot provide the exact figure for the GDP of India in the year 2025 mentioned in the context.", 'source_documents': [Document(id='b3186214-1968-4159-985b-d0d145d67a56', metadata={}, page_content='spotlight story 4\n\nTrump’s tariffs hit India: What it means for GDP, debt and equity markets\n\nET ePaper Logo\n\nSunday, 17 Aug, 2025\n\nExperience Your Economic Times Newspaper, The Digital Way!\n\nStory Image_1\n\nTrump Hints at No Extra Levy on India for Russian Oil Buy\n\nPresident Donald Trump has indicated that the US may not impose secondary tariffs on countries continuing to procure Russian crude oil.\n\nStory Image_2\n\nGST 2.0 to Offer Tr

In [ ]:
result = qa_chain({"query": query}, return_only_outputs=True)

In [ ]:
print(result)

{'result': " The provided context does not explicitly state the current GDP of India. However, it mentions that the tariffs implemented by Trump could have an impact on India's GDP. To find out the current GDP, you may want to check recent news sources or reliable financial databases.", 'source_documents': [Document(id='b3186214-1968-4159-985b-d0d145d67a56', metadata={}, page_content='spotlight story 4\n\nTrump’s tariffs hit India: What it means for GDP, debt and equity markets\n\nET ePaper Logo\n\nSunday, 17 Aug, 2025\n\nExperience Your Economic Times Newspaper, The Digital Way!\n\nStory Image_1\n\nTrump Hints at No Extra Levy on India for Russian Oil Buy\n\nPresident Donald Trump has indicated that the US may not impose secondary tariffs on countries continuing to procure Russian crude oil.\n\nStory Image_2\n\nGST 2.0 to Offer Treatment for Other Pain Points Too\n\nThe proposed goods and services tax (GST) 2.0, hammered out over months of extensive deliberations, will go beyond the m